In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("fr_wiki")

Vocabulary will not be loaded from file.



In [4]:
semiotic.vocab.build(vocab_size = 503,parallel=False, parallel_mode="process")


Alphabet Size: 498



Vocabulary built


In [5]:
semiotic.vocab.alpha

{'e': 2519128,
 'a': 1407511,
 's': 1335796,
 'n': 1309534,
 'i': 1307647,
 'r': 1231139,
 't': 1204908,
 'l': 1048131,
 'o': 976389,
 'u': 923685,
 'd': 761112,
 'c': 610267,
 'p': 497908,
 'm': 494139,
 'é': 422979,
 'g': 232012,
 'v': 224262,
 'h': 199616,
 'f': 198180,
 'b': 187737,
 'q': 130512,
 '1': 106323,
 'y': 79691,
 'x': 77466,
 'à': 73672,
 '0': 71514,
 'è': 66540,
 '9': 65768,
 'j': 59196,
 '2': 57157,
 'k': 46887,
 '8': 32822,
 'w': 27586,
 '3': 26276,
 'z': 25807,
 '7': 25588,
 '5': 25550,
 '4': 24803,
 '6': 23762,
 'ê': 16535,
 'ç': 11036,
 '«': 9662,
 '»': 9385,
 'ô': 7920,
 'â': 7716,
 'î': 6555,
 '□': 5034,
 'ï': 4197,
 'ù': 3839,
 'œ': 3589,
 'û': 2209,
 'á': 1639,
 '°': 1367,
 'ü': 1160,
 'í': 1095,
 'ó': 1079,
 'ö': 926,
 'ë': 837,
 'ō': 837,
 'ä': 690,
 'ł': 474,
 '†': 461,
 'š': 421,
 'ć': 400,
 'č': 313,
 'ñ': 284,
 'ã': 268,
 'ú': 228,
 '×': 200,
 'ū': 188,
 'ā': 186,
 'æ': 171,
 'α': 125,
 '́': 125,
 'ś': 123,
 'å': 122,
 'ń': 121,
 'ž': 113,
 'ș': 112,
 'ă'

In [43]:
import re

def strip_cjk(string):
    p = re.compile(r"[\uac00-\ud7a3]|[\u4e00-\u9FFF]|[\u3040-\u30ff]")
    
    # |re.compile(r"[\u4e00-\u9FFF]")|re.compile(r"[\u3040-\u30ff]")
    filter = p.sub("□", string)
    return filter

In [44]:
strip_cjk("2009年，波音公司(Boeing)在查尔斯顿附近的新厂破土动工时，曾宣扬这里是最先进的制造中心 // 日産自動車は24日、2019年3 // 성 간부 총살설 주민들 사이")

'2009□，□□□□(Boeing)□□□□□□□□□□□□□□□，□□□□□□□□□□□□□□ // □□□□□□24□、2019□3 // □ □□ □□□ □□□ □□'

In [42]:
p = re.compile(r"[\uac00-\ud7a3]|[\u4e00-\u9FFF]|[\u3040-\u30ff]")
cjk = [(k,v) for k,v in semiotic.vocab.freq.items() if not re.search("[\uac00-\ud7a3]|[\u4e00-\u9FFF]|[\u3040-\u30ff]", k)]
cjk

[('e', 1454260),
 ('i', 1307647),
 ('r', 1231139),
 ('t', 1204908),
 ('a', 1165368),
 ('l', 1048131),
 ('u', 923685),
 ('s', 864260),
 ('o', 688249),
 ('c', 610267),
 ('p', 497908),
 ('d', 497415),
 ('m', 494139),
 ('es', 471536),
 ('n', 449616),
 ('é', 422979),
 ('en', 329635),
 ('on', 288140),
 ('de', 263697),
 ('an', 242143),
 ('g', 232012),
 ('v', 224262),
 ('h', 199616),
 ('f', 198180),
 ('b', 187737),
 ('q', 130512),
 ('1', 106323),
 ('y', 79691),
 ('x', 77466),
 ('à', 73672),
 ('0', 71514),
 ('è', 66540),
 ('9', 65768),
 ('j', 59196),
 ('2', 57157),
 ('k', 46887),
 ('8', 32822),
 ('w', 27586),
 ('3', 26276),
 ('z', 25807),
 ('7', 25588),
 ('5', 25550),
 ('4', 24803),
 ('6', 23762),
 ('ê', 16535),
 ('ç', 11036),
 ('«', 9662),
 ('»', 9385),
 ('ô', 7920),
 ('â', 7716),
 ('î', 6555),
 ('ï', 4197),
 ('ù', 3839),
 ('œ', 3589),
 ('û', 2209),
 ('á', 1639),
 ('°', 1367),
 ('ü', 1160),
 ('í', 1095),
 ('ó', 1079),
 ('ö', 926),
 ('ë', 837),
 ('ō', 837),
 ('ä', 690),
 ('ł', 474),
 ('†', 461)

In [ ]:



def cjk_detect(texts):
    # korean
    if re.search("[\uac00-\ud7a3]", texts):
        return "ko"
    # japanese
    if re.search("[\u3040-\u30ff]", texts):
        return "ja"
    # chinese
    if re.search("[\u4e00-\u9FFF]", texts):
        return "zh"
    return None

In [3]:
corpus = semiotic.corpus.train

normalizer = slg.syntagmatic.tokenizer.normalizers.Sequence(semiotic.vocab.config.normalizer)

chain_list = []

alphabet = Counter()

for sent in tqdm(corpus, desc="Chain List & Alphabet:"):
    sent = normalizer.normalize(sent)
    sent = list(sent)
    if sent !=[]:
        chain_list += sent
        alphabet.update(Counter(sent))

In [4]:
reload(temp)

<module 'temp' from '/Users/Gianni/semiolog/wip/temp.py'>

In [5]:
for i in range(5):
    now = datetime.now()
    result = util.multiprocessing(temp.find_best_pair, temp.parallel_chain(chain_list,4), cores = 4)

    parallel = reduce(operator.add, result)
        
    parallel = parallel.most_common()
    print(datetime.now() - now)

0:00:02.978095
0:00:02.831269
0:00:03.196081
0:00:03.024566
0:00:02.903223


In [25]:
for i in range(5):
    now = datetime.now()
    
    sequential = temp.find_best_pair(chain_list)
    sequential = sequential.most_common()
    
    print(datetime.now() - now)

0:00:05.527317
0:00:05.145561
0:00:05.272142
0:00:05.325645
0:00:05.272945


In [19]:
 parallel == sequential

True

In [3]:
semiotic.vocab.build(vocab_size = 1627)


Vocabulary built


In [4]:
semiotic.vocab.build(vocab_size = 1627, parallel=True, parallel_mode="process")


Vocabulary built


In [3]:
def chain_list_alpha(self, normalizer, corpus_sents, progress_bar = False):
    chain_list = []
    alphabet = Counter()
    for sent in corpus_sents:
        sent = normalizer.normalize(sent)
        sent = list(sent)
        if sent !=[]:
            chain_list += sent
            alphabet.update(Counter(sent))
    return chain_list, alphabet

4

In [3]:
from tqdm.notebook import tqdm
import time
from temp import chain_list_i, chain_list
from functools import partial

In [4]:
def parallel_chain(chain, n_of_parts):
    """
    Breaks the chain in n chunks to compute best pair of terms. Chunks are overlapping by one term, so as no pair of terms is lost due to the break.
    """
    if not isinstance(chain,list):
        chain = list(chain)
    chunk_size = int(len(chain) / n_of_parts)+1
    for i in range(0, len(chain), chunk_size):
        yield chain[i : i + chunk_size +1]

In [5]:
normalizer = eval(
                f"slg.syntagmatic.tokenizer.normalizers.Sequence({slg.util.if_none_disable(semiotic.config.vocabulary.normalizer)})"
            )

In [6]:
bla = chain_list(normalizer,semiotic.corpus.train)

In [ ]:
%%time

bla_p = slg.util.multiprocessing(partial(chain_list, normalizer),parallel_chain(semiotic.corpus.train,4),cores=4)

CPU times: user 303 ms, sys: 190 ms, total: 493 ms
Wall time: 5.57 s


In [ ]:
%%time

bla_t = slg.util.multithreading(partial(chain_list_i, normalizer),semiotic.corpus.train,cores=4)